In [1]:
#install the required libraries
!pip install sentence-transformers
!pip install pandas

In [2]:
#import required libraries
import pandas as pd
from sentence_transformers import SentenceTransformer, util

In [3]:
#load the CSV file
try:
  df = pd.read_csv('/content/sample_data/Fashion Dataset v2.csv')
except FileNotFoundError:
  print("Error: 'Fashion Dataset v2.csv' not found in /content/sample_data/")
  exit()

In [4]:
#check the size and other attributes of the dataset
df.size

156354

In [5]:
df.head()

,p_id,name,products,price,colour,brand,img,ratingCount,avg_rating,description,p_attributes
0,17048614,Khushal K Women Black Ethnic Motifs Printed Ku...,"Kurta, Palazzos, Dupatta",5099.0,Black,Khushal K,http://assets.myntassets.com/assets/images/170...,4522.0,4.418399,Black printed Kurta with Palazzos with dupatta...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
1,16524740,InWeave Women Orange Solid Kurta with Palazzos...,"Kurta, Palazzos, Floral Print Dupatta",5899.0,Orange,InWeave,http://assets.myntassets.com/assets/images/165...,1081.0,4.119334,Orange solid Kurta with Palazzos with dupatta<...,"{'Add-Ons': 'NA', 'Body Shape ID': '443,333,32..."
2,16331376,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,"Kurta, Trousers, Dupatta",4899.0,Navy Blue,Anubhutee,http://assets.myntassets.com/assets/images/163...,1752.0,4.161530,Navy blue embroidered Kurta with Trousers with...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
3,14709966,Nayo Women Red Floral Printed Kurta With Trous...,"Kurta, Trouser, Dupatta",3699.0,Red,Nayo,http://assets.myntassets.com/assets/images/147...,4113.0,4.088986,Red printed kurta with trouser and dupatta<br>...,"{'Add-Ons': 'NA', 'Body Shape ID': '333,424', ..."
4,11056154,AHIKA Women Black & Green Printed Straight Kurta,Kurta,1350.0,Black,AHIKA,http://assets.myntassets.com/assets/images/110...,21274.0,3.978377,"Black and green printed straight kurta, has a ...","{'Body Shape ID': '424', 'Body or Garment Size..."


In [6]:
#select relevant columns where we have the product descriptions
product_df = df[['p_id', 'name', 'description','img']]

In [8]:
product_df.size

28428

In [9]:
product_df.head()

,name,description
0,Khushal K Women Black Ethnic Motifs Printed Ku...,Black printed Kurta with Palazzos with dupatta...
1,InWeave Women Orange Solid Kurta with Palazzos...,Orange solid Kurta with Palazzos with dupatta<...
2,Anubhutee Women Navy Blue Ethnic Motifs Embroi...,Navy blue embroidered Kurta with Trousers with...
3,Nayo Women Red Floral Printed Kurta With Trous...,Red printed kurta with trouser and dupatta<br>...
4,AHIKA Women Black & Green Printed Straight Kurta,"Black and green printed straight kurta, has a ..."


In [10]:
#initialize the Sentence Transformer model
model = SentenceTransformer('all-mpnet-base-v2')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [11]:
#create embeddings for product descriptions
product_df['embeddings'] = product_df['description'].apply(lambda x: model.encode(x))

KeyboardInterrupt: 

In [ ]:
product_df.head()

In [ ]:
#now create a function that can answer questions based on the product descriptions
def answer_product_question(question):
  question_embedding = model.encode(question)
  similarities = product_df['embeddings'].apply(lambda x: util.cos_sim(question_embedding, x).item())
  most_similar_index = similarities.argmax()

  #return the most relevant product details
  product_name = product_df.iloc[most_similar_index]['name']
  product_description = product_df.iloc[most_similar_index]['description']
  product_img = product_df.iloc[most_similar_index]['img']
  similarity_score = similarities.max()

  return {
    "product_name": product_name,
    "product_description": product_description,
    "product_img": product_img,
    "similarity_score": similarity_score
  }
